In [5]:
# %% Importação de Bibliotecas
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import itertools  # Para gerar combinações de configurações
import os        # Para manipulação de diretórios
from sklearn.metrics import mean_squared_error

# %% Leitura e Preparação dos Dados
# Substitua "dataset.csv" pelo caminho do seu arquivo de dados
data = pd.read_csv("dataset.csv")

# Convertendo a coluna 'id' para datetime e definindo como índice
# Certifique-se de que a coluna 'id' existe e está no formato correto
try:
    data.index = pd.to_datetime(data['id'], errors='ignore')
    data.index = data['id'].apply(lambda x: pd.to_datetime(str(x).split('.')[0], errors='ignore'))
except Exception as e:
    raise ValueError(f"Erro ao converter a coluna 'id' para datetime: {e}")

# Criando um DataFrame para armazenar os sinais denoised
variables = pd.DataFrame(index=data.index)

# %% Função para Suavização usando Wavelet Transform
def wavelet_denoising(signal, wavelet='db4', level=3, threshold_mode='soft', signal_mode='symmetric'):
    """
    Aplica denoising a um sinal utilizando a transformada wavelet.

    Parameters:
    - signal: array_like, sinal original.
    - wavelet: str, tipo de wavelet a ser usada.
    - level: int, nível de decomposição.
    - threshold_mode: str, modo de thresholding ('soft', 'hard', 'greater', 'garrote').
    - signal_mode: str, modo de extensão de sinal ('symmetric', 'periodic', 'zero', 'constant', 'reflect', etc.).

    Returns:
    - reconstructed_signal: array_like, sinal denoised.
    """
    # Decomposição do sinal
    coeffs = pywt.wavedec(signal, wavelet, mode=signal_mode, level=level)
    
    # Estimativa do ruído
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    uthresh = sigma * np.sqrt(2 * np.log(len(signal)))
    
    # Aplicação do thresholding
    if threshold_mode in ['soft', 'hard']:
        coeffs[1:] = [pywt.threshold(i, value=uthresh, mode=threshold_mode) for i in coeffs[1:]]
    elif threshold_mode == 'greater':
        # Thresholding 'greater': zero coefficients below threshold
        coeffs[1:] = [pywt.threshold(i, value=uthresh, mode='hard') for i in coeffs[1:]]
        coeffs[1:] = [i * (np.abs(i) > uthresh) for i in coeffs[1:]]
    elif threshold_mode == 'garrote':
        # Implementando o thresholding 'garrote' manualmente
        coeffs_new = []
        for i in coeffs[1:]:
            mask = np.abs(i) > uthresh
            coeffs_new.append(np.sign(i) * (np.abs(i) - uthresh) * mask)
        coeffs[1:] = coeffs_new
    else:
        raise ValueError(f"Modo de thresholding '{threshold_mode}' não reconhecido.")
    
    # Reconstrução do sinal
    reconstructed_signal = pywt.waverec(coeffs, wavelet, mode=signal_mode)
    
    # Ajustar o tamanho do sinal reconstruído se necessário
    reconstructed_signal = reconstructed_signal[:len(signal)]
    
    return reconstructed_signal

# %% Definição das Configurações a Testar

# Lista ampliada de wavelets disponíveis no PyWavelets
wavelets = [
    # Daubechies
    'db1', 'db3', 'db5',  'db7',  'db9',
    'db11',  'db13', 'db15', 'db17', 'db19',
    # Symlets
    'sym2',  'sym4', 'sym6',  'sym8', 'sym10',
    'sym12', 'sym14', 'sym16',  'sym18', 'sym20',
    # Coiflets
    'coif1', 'coif3', 'coif5',
    # Biorthogonais
    'bior1.1', 'bior1.5',
    'bior2.2', 'bior2.6', 'bior2.8',
    'bior3.1', 'bior3.5', 'bior3.7', 'bior3.9',
    'bior4.4', 'bior6.8',
    # Reverse Biorthogonais
    'rbio1.1',  'rbio1.5',
    'rbio2.2', 'rbio2.6', 'rbio2.8',
    'rbio3.1', 'rbio3.5', 'rbio3.7', 'rbio3.9',
    'rbio4.4', 'rbio6.8',
    # Haar e Outros
    'haar', 'haar', 'dmey'
]

# Níveis de decomposição a serem testados
levels = [3, 4]

# Modos de Thresholding
threshold_modes = ['soft', 'garrote']

# Modos de Extensão de Sinal
signal_modes = ['symmetric', 'smooth', 'periodization']

# Gerando todas as combinações possíveis
configuracoes = list(itertools.product(wavelets, levels, threshold_modes, signal_modes))

# Opcional: Limitar o número de configurações para evitar excesso de plots
# Por exemplo, selecionar apenas as primeiras 50 configurações
# configuracoes = configuracoes[:50]

# %% Criação de Diretório para Salvar os Gráficos e Resultados
# Define um diretório para salvar os gráficos e métricas
output_dir = "wavelet_denoising_plots_comparacao"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Adicionando uma subpasta para métricas
metrics_dir = os.path.join(output_dir, "metrics")
if not os.path.exists(metrics_dir):
    os.makedirs(metrics_dir)

# %% Colunas a Serem Processadas
colunas = ['ws100']

# %% Inicialização para Armazenar Resultados das Métricas
resultados_mse = []

# %% Teste das Configurações e Plotagem da Comparação
for idx, (wavelet, level, threshold_mode, signal_mode) in enumerate(configuracoes, start=1):
    print(f"Testando Configuração {idx}: Wavelet={wavelet}, Level={level}, Threshold Mode={threshold_mode}, Signal Mode={signal_mode}")
    
    # Aplicação do Wavelet Denoising às Colunas Desejadas
    for col in colunas:
        if col in data.columns:
            try:
                denoised_signal = wavelet_denoising(
                    data[col].values, 
                    wavelet=wavelet, 
                    level=level, 
                    threshold_mode=threshold_mode, 
                    signal_mode=signal_mode
                )
                variables[f'{col}_wavelet_{idx}'] = denoised_signal
            except Exception as e:
                print(f"Erro ao processar a coluna {col} com a configuração {idx}: {e}")
                continue
        else:
            raise ValueError(f"A coluna {col} não existe no dataset.")
    
    # Plotagem da Comparação para Cada Coluna
    for col in colunas:
        if f'{col}_wavelet_{idx}' in variables.columns:
            plt.figure(figsize=(20, 12))
            plt.plot(data.index[1500:2500], data[col][1500:2500], label='Original', alpha=0.7)
            plt.plot(variables.index[1500:2500], variables[f'{col}_wavelet_{idx}'][1500:2500], label='Wavelet Denoised', linestyle='--')
            plt.title(f'Comparação entre Original e Wavelet Denoised para {col}\n'
                      f'Configuração {idx}: Wavelet={wavelet}, Level={level}, Threshold Mode={threshold_mode}, Signal Mode={signal_mode}')
            plt.xlabel('Data')
            plt.ylabel('Valor')
            plt.legend()
            plt.tight_layout()
            
            # Salvar o gráfico em arquivo
            plot_filename = f"{output_dir}/comparacao_{col}_config_{idx}.png"
            plt.savefig(plot_filename)
            plt.close()  # Fecha a figura para liberar memória
            print(f"Gráfico salvo em: {plot_filename}")
            
            # Cálculo do MSE
            mse = mean_squared_error(data[col].values, variables[f'{col}_wavelet_{idx}'])
            resultados_mse.append({
                'Configuração': idx,
                'Wavelet': wavelet,
                'Level': level,
                'Threshold Mode': threshold_mode,
                'Signal Mode': signal_mode,
                'Coluna': col,
                'MSE': mse
            })

    # Progressão a cada 10 configurações
    if idx % 10 == 0:
        print(f"{idx} configurações processadas.")

# %% Salvando os Resultados das Métricas
df_mse = pd.DataFrame(resultados_mse)
mse_filename = os.path.join(metrics_dir, "mse_resultados.csv")
df_mse.to_csv(mse_filename, index=False)
print(f"Resultados das métricas MSE salvos em: {mse_filename}")
print("Processamento concluído!")


/tmp/ipykernel_3088/3451176622.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data.index = pd.to_datetime(data['id'], errors='ignore')
/tmp/ipykernel_3088/3451176622.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data.index = data['id'].apply(lambda x: pd.to_datetime(str(x).split('.')[0], errors='ignore'))


Testando Configuração 1: Wavelet=db1, Level=3, Threshold Mode=soft, Signal Mode=symmetric
Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_1.png
Testando Configuração 2: Wavelet=db1, Level=3, Threshold Mode=soft, Signal Mode=smooth
Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_2.png
Testando Configuração 3: Wavelet=db1, Level=3, Threshold Mode=soft, Signal Mode=periodization
Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_3.png
Testando Configuração 4: Wavelet=db1, Level=3, Threshold Mode=garrote, Signal Mode=symmetric
Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_4.png
Testando Configuração 5: Wavelet=db1, Level=3, Threshold Mode=garrote, Signal Mode=smooth
Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_5.png
Testando Configuração 6: Wavelet=db1, Level=3, Threshold Mode=garrote, Signal Mode=periodization
Gráfico salvo em: wavelet_denoisin

/tmp/ipykernel_3088/3451176622.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  variables[f'{col}_wavelet_{idx}'] = denoised_signal


Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_101.png
Testando Configuração 102: Wavelet=db17, Level=3, Threshold Mode=garrote, Signal Mode=periodization


/tmp/ipykernel_3088/3451176622.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  variables[f'{col}_wavelet_{idx}'] = denoised_signal


...

Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_574.png
Testando Configuração 575: Wavelet=dmey, Level=4, Threshold Mode=garrote, Signal Mode=smooth


/tmp/ipykernel_3088/3451176622.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  variables[f'{col}_wavelet_{idx}'] = denoised_signal


Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_575.png
Testando Configuração 576: Wavelet=dmey, Level=4, Threshold Mode=garrote, Signal Mode=periodization


/tmp/ipykernel_3088/3451176622.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  variables[f'{col}_wavelet_{idx}'] = denoised_signal


Gráfico salvo em: wavelet_denoising_plots_comparacao/comparacao_ws100_config_576.png
Resultados das métricas MSE salvos em: wavelet_denoising_plots_comparacao/metrics/mse_resultados.csv
Processamento concluído!
